# Import & Load Vulnerability Data

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open("../../../settings.ini"))

sys.path.insert(0, config.get("PATHS", "libs_path"))
engine_path = config.get("DATABASE", "engine_path")

### Main params

In [2]:
remote_path = ""
local_path = "../data_final/"

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [4]:
#Open unzipped file
df = pd.read_csv(local_path + "ive_junaeb.csv", delimiter=",")
# df = df[['year','comuna_id','sex_id','rate_comuna','rate_region','rate_country','count','age_range_id']]
# df = df.astype({"year":"int", "comuna_id": "int",'sex_id':'int','rate_comuna':'float','rate_region':'float','rate_country':'float', 'age_range_id': 'float'})
list(df)

['rbd',
 'administration_id',
 'zone_id',
 'comuna_id',
 'education_id',
 'year',
 'priority_id',
 'total']

### Ingest

In [5]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, "childhood", "fact_vulnerability")

DROP TABLE IF EXISTS childhood.fact_vulnerability;
CREATE TABLE "childhood"."fact_vulnerability" (
"rbd" INTEGER,
  "administration_id" INTEGER,
  "zone_id" INTEGER,
  "comuna_id" INTEGER,
  "education_id" INTEGER,
  "year" INTEGER,
  "priority_id" INTEGER,
  "total" INTEGER
)
COPY "childhood"."fact_vulnerability" ("rbd","administration_id","zone_id","comuna_id","education_id","year","priority_id","total") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Add Indices to foreign keys

In [6]:
engine.execute("""
CREATE INDEX fact_vulnerability_comuna_id_index 
ON childhood.fact_vulnerability (comuna_id)
""")

engine.execute("""
CREATE INDEX fact_vulnerability_year_index 
ON childhood.fact_vulnerability ("year");
""")

### Relational Tables

In [7]:
d2 = download_file(remote_path, local_path, "priority.csv")
print (inline_table_xml(d2, "name_en", "id", "name_es"))

Already downloaded. Using: ../data_final/priority.csv

<InlineTable alias="name_en">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">0</Value>
      <Value column="description">Sin información</Value>
      <Value column="es_description">Sin información</Value>
    </Row>
  <Row>
      <Value column="id">1</Value>
      <Value column="description">Primera prioridad</Value>
      <Value column="es_description">Primera prioridad</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Segunda prioridad</Value>
      <Value column="es_description">Segunda prioridad</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Tercera prioridad</Value>
      <Value column="es_description">Tercera prioridad</Value>
    </Row>
  <Row>
      <Value 